In [3]:
from pymongo import MongoClient
import os
from collections import Counter

#ignore mongo warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

#Database definitions and cleaning
client = MongoClient()
myClient = MongoClient('localhost', 27017)
myDB = myClient["marketpulzz"]
myColl = myDB["tutorial"]
myColl.remove({})

#read input
directory = input("Enter the directory")

class WordsCounter:
    
    def __init__(self, directory):
        self.directory = directory
        self.words_counter = Counter()

    #Main function iterating and counting on files in dir
    def count(self):      
        for filename in os.listdir(self.directory):
            if not filename.startswith('.'):
                filepath = os.path.join(directory, filename)
                self.read_file(filepath)
        print(self.words_counter.most_common())

    #open file and pass it to counter
    def read_file(self, filename):
        with open(filename, 'r') as f:
            self.count_words(f)
        
    #Count words in a file and write to db
    def count_words(self, file):
        index = 0
        for line in file:
            for word in line.split():
                self.words_counter[word]+=1
                self.write_to_db(word, self.words_counter[word], file.name, index)
                index+=1
    
    #update or insert whether the word exists
    def write_to_db(self, word, word_count, filename, index):
        if myColl.find({"word": word}).count() > 0:
            myColl.update({"word": word}, {"$inc": {"count":1}, "$push" : {"locations" : {"filename": filename, "index" : index}}})
        else:
            myColl.insert_one({"word" : word, "locations": [{"filename": filename, "index" : index}], "count" : 1})
            
                
myCounter = WordsCounter(directory)
myCounter.count()

Enter the directory/Users/leon/Desktop/files
[('word1', 3), ('word', 1)]
